# Organización de Datos (9558)

## Trabajo Práctico N° 2

#### Fecha: 2017-06-22

#### Integrantes:


#### Repo:
[link]()

---
*toc*

---
#### Enunciado

El segundo TP es una competencia de Machine Learning en donde cada grupo debe intentar predecir la duración de los viajes en base a los datos de los mismos. La competencia se desarrolla en la plataforma de Kaggle, se provee un archivo "train.csv" que debe ser usado para entrenar un modelo de Machine Learning y un archivo "test.csv" que tiene los datos de los viajes a predecir. Adicionalmente pueden usarse los datos de las estaciones, del status de cada estación minuto a minuto y la información meteorológica.

ADVERTENCIA MUY IMPORTANTE: Dado que los datos son públicos los resultados de la competencia ya se saben, son parte de los datos del TP1, es fundamental que ningún grupo suba a Kaggle submissions que usan esta información ya que distorsiona el score de la competencia y una vez subido un submission es IMPOSIBLE ELIMINARLO. Los submissions
deben generarse en base a un modelo de machine learning y nunca en base a los resultados que ya se conocen.

TL;DR: NO SUBIR A KAGGLE SUBMISSIONS QUE HACEN TRAMPA, ES IMPOSIBLE BORRARLOS!!!

Los grupos deberán probar distintos algoritmos de Machine Learning para predecir la duración de los viajes en base a los datos de los mismos. A medida que los grupos realicen pruebas deben realizar el correspondiente submit en Kaggle para evaluar el resultado de los mismos. Al finalizar la competencia el grupo que mejor resultado tenga obtendrá 10 puntos para cada uno de sus integrantes que podrán ser usados en el examen por promoción o segundo recuperatorio.

Requisitos para la entrega del TP2:
1. El TP debe programarse en Python o R
- Debe entregarse una carpeta con el informe de algoritmos probados, algoritmo final utilizado, transformaciones realizadas a los datos, feature engineering, etc.
- El grupo debe presentar el TP en una computadora en la fecha indicada por la cátedra, el TP debe correr en un lapso de tiempo razonable (inferior a 1 hora) y generar un submission válido que iguale el mejor resultado obtenido por el grupo en Kaggle.

El TP2 se va a evaluar en función del siguiente criterio:
1. Cantidad de trabajo (esfuerzo) del grupo: ¿Probaron muchos algoritmos? ¿Hicieron un buen trabajo de pre-procesamiento de los datos y feature engineering?
- Resultado obtenido en Kaggle (obviamente cuanto mejor resultado mejor nota)
- Presentación final del informe, calidad de la redacción, uso de información obtenida en el TP1, conclusiones presentadas.
- Performance de la solución final.


ADVERTENCIA IMPORTANTE #2: Bajo ningún concepto debe interpretarse que es necesario finalizar el TP1 para poder comenzar el TP2, quienes incurran en este error se encontrarán que el tiempo necesario para desarrollar el TP2 es insuficiente. Es fundamental, imprescindible y vital comenzar el desarrollo del TP2 en forma paralela al TP1 para evitar problemas en el cumpimiento de las fechas de entrega.

---
#### [Descipción](https://inclass.kaggle.com/c/san-francisco-biking)

El propósito de esta competencia es predecir la duración de los viajes.

El objetivo de esta competencia es predecir la duración de los viajes de acuerdo a los datos de los mismos, la información meteorológica, etc.

Pueden usarse los siguientes archivos disponibles en:

https://www.kaggle.com/benhamner/sf-bay-area-bike-share

    .status
    .station 
    .weather

Para los viajes debe usarse el archivo train.csv provisto par entrenar el modelo y el archivo test.csv tiene los datos de los viajes a predecir.

IMPORTANTE: El archivo trips.csv del TP anterior tiene los resultados que queremos predecir, estos resultados NO DEBEN usarse de ninguna forma para el entrenamiento del modelo. No subir a Kaggle ningún submission que sea generado a partir de estos datos ya que afecta la evaluación de la competencia y NO ES POSIBLE ELIMINAR SUBMISSIONS. 

---
#### [Evaluación y Formato de Entrega](https://inclass.kaggle.com/c/san-francisco-biking/details/evaluation)

La evaluación es por Mean Squared Error es decir la diferencia de cuadrados entre la duración real del viaje y la duración que se predice mediante el modelo de Machine Learning usado.

El formato de submission es un archivo .csv de la forma:

    id,duration
    1,1
    8,8
    9,9 10
    10,10 9
    etc.

(Acá no me queda claro cómo es el formato del archivo con las predicciones). -mk

---
#### Conclusiones del TP 1

*conclusiones*

> Los usuarios del serivicio, en su mayoria, lo utiliza para trasladarse al trabajo.

> Ya conocemos cuáles son los datos a filtrar.

---
(Cómo manejamos los datos? Si vamos a utilizar un API (por ejemplo tensorflow) Cómo se utiliza? Todos sabemos usarlo? Qué complicaciones puede traer? Qué puede facilitar? Hay otras alternativas? Cúales? Son mas simples?) -mk

#### Introduciendo a la API

[tensorflow](https://www.tensorflow.org/get_started/get_started)

(En el link, hay explicaciones de las estructuras utilizadas en tensorflow.)

Asumiendo que utilizamos tensorflow, podemos armar un modelo basico.

__Ejemplo de una parte del modelo__

Un dato entra, separamos las partes (o dimensiones) que consideramos importantes, realizamos cálculos con ellos y devolvemos un resultado.

Una parte del algoritmo (un *nodo* en tensorflow), podría tomar como parámetros a las id's de las estaciones y devolver la distancia entre ellas. Luego la distancia, irá por otras partes del modelo que serán utilizadas para determinar el resultado final que es la duración del viaje.

(Acá van las consideraciones de la api, (que estructuras de datos estan disponibles, como se pueden utilizar y como las VAMOS a utilizar). Despues en __plan__, ya juntamos todas las consideraciones y formamos el plan) -mk

---
#### Sets de Datos

Los sets de datos son:

>[Entrenamiento y testeo](https://inclass.kaggle.com/c/san-francisco-biking/data)

>[Set de datos del TP1](https://www.kaggle.com/benhamner/sf-bay-area-bike-share) (excluyendo a trips.csv)

---
(quitar esta sección. Está sólo para aclarar los métodos para obtener una mejor performance en tanto a las predicciones del algoritmo. Pero eso se debe realizar luego de armar el modelo)
##### Dividiendo al set de entrenamiento

__Bootstrapping:__

1. Tomamos n muestras del set de entrenamiento (con reemplazo, el dato puede repetirse) y corremos el algoritmo para esas n muestras.
2. Con esto generamos n modelos distintos.
3. Luego corremos el set de testing con esos n modelos, y llegamos a un resultado por votación (promediando).

La técnica de bagging disminuye la posibilidad de overfitting

Como cada clasificador no ve la totalidad de los registros del set de entrenamiento ninguno de los n clasificadores individuales puede sobre-ajustar.

Registros OOB (Out of Bag), es decir fuera del bagging se usan para ver la precisión del algoritmo. (como se fuera de un set de validación).

Promedio de precisión de los registros OOB se puede usar para analizar la precisión del ensamble entero, y obtener los mejores hiper-parámetros.

__k-fold Cross Validation:__

1. Dividir la muestra en k conjuntos 
2. Para i = 1..k: entrenar en todos menos i-ésimo conjunto, validar sobre el i-ésimo conjunto 
3. Performance: promedio de las k iteraciones
    
Esto nos permite comparar la performance de distintos modelos para nuestro set de datos, fijar parametros de ajuste, etc.

>__Luego de definido el modelo, podemos entrenarlo con nuestro set de entrenamiento completo.__

---
##### Datos a filtrar

Hechemos un vistazo del set de datos.

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('./data/trip_train.csv', low_memory=False)
test = pd.read_csv('./data/trip_test.csv', low_memory=False)

In [13]:
train.sample(3)

,id,duration,s_date,ss_name,ss_id,e_date,es_name,es_id,b_id,sub_type,z_code
479991,438369,273,2014-09-04 14:56:00,Clay at Battery,41,2014-09-04 15:00:00,Temporary Transbay Terminal (Howard at Beale),55,385,Subscriber,94105
396013,568857,548,2014-12-07 20:00:00,Mechanics Plaza (Market at Battery),75,2014-12-07 20:09:00,Powell Street BART,39,677,Subscriber,94107
201689,262617,162,2014-04-28 06:12:00,Townsend at 7th,65,2014-04-28 06:14:00,San Francisco Caltrain 2 (330 Townsend),69,610,Subscriber,94014


In [12]:
test.sample(3)

,id,s_date,ss_name,ss_id,e_date,es_name,es_id,b_id,sub_type,z_code
111894,369495,2014-07-18 09:07:00,San Francisco Caltrain (Townsend at 4th),70,2014-07-18 09:14:00,Howard at 2nd,63,481,Subscriber,95112
61620,201363,2014-03-04 15:04:00,Market at Sansome,77,2014-03-04 15:18:00,Townsend at 7th,65,592,Subscriber,94925
101469,261849,2014-04-26 15:00:00,Embarcadero at Folsom,51,2014-04-26 15:15:00,San Francisco Caltrain (Townsend at 4th),70,312,Subscriber,94105


In [8]:
# Comparamos los sets de datos
print float(train.shape[0] / test.shape[0])

4.0


En un principio los datos de entrenamiento son 4 veces mayores a los de testeo.

Algo importante de notar de esto, es que cuando el algoritmo de predicción esté armado y preparado, los datos que ingresaremos serán los de testeo. Digo esto porque, si el algoritmo aprende a predecir duración en función de distancias, deberá tener en cuenta que primero se debe calcular la distancia entre las estaciones (ya no es parte del dato de prueba en sí).

Los nombres de las estaciones no son necesarios ya que se conocen sus identificaciones.

El tipo de suscripción se puede convertir a un número. Además sólo hay dos tipos de suscripción.

Las identificaciones de los viajes tampoco son necesarias ya que no aportan informacion útil.

(Zip-code, en este caso no me acuerdo que era, pero creo se puede sacar. Ademas hay algunos datos con NaN)

(El set de testeo (creo) que __NO__ se debe tocar (en cuanto a valores NaN y eso) aunque si el formato de fechas, etc)

In [ ]:
renombres = {'subscription_type': 'sub_type',
             'start_station_name': 'ss_name',
             'end_station_name': 'es_name',
             'start_station_id': 'ss_id',
             'end_station_id': 'es_id',
             'start_date': 's_date',
             'end_date': 'e_date',
             'zip_code': 'z_code',
             'bike_id': 'b_id'}

train.rename(columns=renombres, inplace=True)
test.rename(columns=renombres, inplace=True)

In [9]:
# Convirtiendo el formato de fecha
train.s_date = pd.to_datetime(train.s_date, format='%m/%d/%Y %H:%M')
train.e_date = pd.to_datetime(train.e_date, format='%m/%d/%Y %H:%M')
test.s_date = pd.to_datetime(test.s_date, format='%m/%d/%Y %H:%M')
test.e_date = pd.to_datetime(test.e_date, format='%m/%d/%Y %H:%M')

In [15]:
#Quitamos los datos invalidos
# (los NaN y eso)

In [ ]:
#Quitamos 

Antes de continuar a la siguiente sección, vemos como es la relación entre los sets de datos

In [14]:
# Comparamos los sets de datos luego de filtrarlos
print float(train.shape[0] / test.shape[0])

4.0


---
#### Análisis Extenso

(Hasta este punto ya se vieron los datos. En esta seccion determinamos cuales son los features importantes y todo lo necesario para tener un plan concreto)

Hasta ahora tenemos conociemiento de: las conclusiones del TP1, el API a utilizar y los datos. En esta sección vamos a ver las ultimas consideraciones.


Como evaluamos a los resultados? (me refiero a las herramienta que provee tensoflow. cual es?)

Como determinamos los tamaños de los sets de datos? (Esto es un hiperparametro y debe testearse luego de tener el algoritmo implementado, y se utilizara la mejor configuracion que es la que da mejores resultados)



---
#### Plan

El problema que se plantea, es de regresión ya que tenemos que predecir la duración de un viaje en minutos, entónces dado un *viaje* (de prueba), debemos devolver un posible valor de duración de dicho *viaje*.

*planteo del algoritmo a utilizar*

La predicción deberá ser un valor mayor a 0.

*plan*

>Primero, creemos que sería útil visualizar los datos en crudo, para ver como se comportan. Es decir, analizar la duracion de los viajes dependiendo de las estaciones o ciudades (esto deberia estar en las __conclusiones del TP1__) o cosas por el estilo.

>Para lograr esto, aplicamos un método de reducción de dimensiones.

>(algunos comentarios: aplicar SVD o PCA y ver los valores singulares no creo que aporte mucho. Hacer eso, nos daria las *dimensiones* que son mas importantes o representativas de los datos. Yo diria que esas *dimensiones* serían las que varian menos, por ejemplo la *ss_id* o *sub_type* ya que tienen pocas variaciones y practicamente todos los datos tienen esa información. Pero ahora me doy cuenta que tal vez se pueda inferir que el tipo de suscripción puede alterar (o se ve un patrón en) el cambio de duración del viaje) -mk

>(Cómo se implementan estas cosas? Qué hacemos con los strings, etc.? Me acuerdo que al principio de la cursada vimos cosas como, convertir esos tipos de datos a valores numéricos y eso pero, es del todo necesario?, los hacemos? cómo? ....) -mk

---
#### Ejecución del plan

In [7]:
#stations 
#stations = pd.read_csv('./data/station.csv', low_memory=False)
#weather = pd.read_csv('./data/weather.csv', low_memory=False)

In [13]:
test.head(3)

,id,s_date,ss_name,ss_id,e_date,es_name,es_id,b_id,sub_type,z_code
0,504737,2014-10-18 11:25:00,Embarcadero at Sansome,60,2014-10-18 12:12:00,Powell at Post (Union Square),71,426,Customer,77009
1,530846,2014-11-05 13:00:00,Embarcadero at Folsom,51,2014-11-05 13:09:00,Broadway St at Battery St,82,454,Subscriber,94132
2,813140,2015-06-18 17:34:00,San Francisco Caltrain (Townsend at 4th),70,2015-06-18 17:37:00,2nd at Townsend,61,370,Subscriber,94107


In [12]:
train.head(3)

,id,duration,s_date,ss_name,ss_id,e_date,es_name,es_id,b_id,sub_type,z_code
0,907649,396,2015-08-27 08:36:00,Harry Bridges Plaza (Ferry Building),50,2015-08-27 08:43:00,Embarcadero at Sansome,60,187,Subscriber,94602
1,384043,636,2014-07-28 22:06:00,Market at 10th,67,2014-07-28 22:17:00,Washington at Kearny,46,417,Subscriber,94133
2,316176,334,2014-06-09 08:42:00,Market at Sansome,77,2014-06-09 08:47:00,2nd at Folsom,62,281,Subscriber,94107


4
